In [1]:
from pathlib import Path

import catboost as cb
import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
data_path = Path("../data/")
seed = 42

In [3]:
def balanced_log_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    n0, n1 = np.bincount(y_true.astype(int))
    w0 = 1 / (n0 / len(y_true))
    w1 = 1 / (n1 / len(y_true))

    l0 = -w0 / n0 * np.sum(np.where(y_true == 0, 1, 0) * np.log(1 - y_pred))
    l1 = -w1 / n1 * np.sum(np.where(y_true != 0, 1, 0) * np.log(y_pred))

    return (l0 + l1) / (w0 + w1)


In [4]:
# read data
train_df = pd.read_csv(data_path / "train.csv")
test_df = pd.read_csv(data_path / "test.csv")
greeks_df = pd.read_csv(data_path / "greeks.csv")

In [5]:
# some columns have trailing spaces
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
feature_cols = train_df.columns.tolist()[1:-1]

In [6]:
# training
oof = np.zeros(len(train_df))
skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
test_probs = []

In [7]:
fold = 1
for train_idx, val_idx in skf.split(train_df, greeks_df.iloc[:, 1:-1]):

    print(f"Fold-{fold}".center(110, "-"))
    fold += 1

    X_train, y_train = (
        train_df.loc[train_idx, feature_cols],
        train_df.loc[train_idx, "Class"],
    )

    X_val, y_val = (
        train_df.loc[val_idx, feature_cols],
        train_df.loc[val_idx, "Class"],
    )

    # params = {
    #     "iterations": 1178,
    #     "use_best_model": True,
    #     "learning_rate": 0.02307884135288726,
    #     "l2_leaf_reg": 8,
    #     "random_strength": 2,
    #     "bagging_temperature": 7.043228545140888,
    #     "grow_policy": "Lossguide",
    #     "auto_class_weights": "Balanced",
    #     "od_type": "IncToDec",
    #     "od_wait": 36,
    # }
    params = {
        "iterations": 10000,
        "learning_rate": 0.005,
        "early_stopping_rounds": 1000,
        "auto_class_weights": "Balanced",
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass:use_weights=False",
        "random_seed": 42,
        "use_best_model": True,
        "l2_leaf_reg": 1,
        "max_ctr_complexity": 15,
        "max_depth": 10,
        "grow_policy": "Lossguide",
        "max_leaves": 64,
        "min_data_in_leaf": 40,
    }
    
    model = cb.CatBoostClassifier(**params)
    model.fit(
        X_train, y_train, eval_set=[(X_val, y_val)], cat_features=["EJ"], verbose=1000
    )

    preds = model.predict_proba(X_val)[:, 1]
    oof[val_idx] = preds

    # make test predictions
    test_preds = model.predict_proba(test_df.iloc[:, 1:])
    test_probs.append(test_preds)

print(f"OOF score: {balanced_log_loss(train_df['Class'], oof):.4f}".center(110, "-"))


----------------------------------------------------Fold-1----------------------------------------------------
0:	learn: 0.6906482	test: 0.6912274	best: 0.6912274 (0)	total: 62.2ms	remaining: 10m 21s
1000:	learn: 0.1061947	test: 0.2128374	best: 0.2128374 (1000)	total: 3.59s	remaining: 32.2s
2000:	learn: 0.0252801	test: 0.1843363	best: 0.1838559 (1943)	total: 7.04s	remaining: 28.2s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.1838559036
bestIteration = 1943

Shrink model to first 1944 iterations.
----------------------------------------------------Fold-2----------------------------------------------------
0:	learn: 0.6908401	test: 0.6908988	best: 0.6908988 (0)	total: 3ms	remaining: 30s
1000:	learn: 0.1083615	test: 0.2026649	best: 0.2026576 (999)	total: 3.41s	remaining: 30.7s
2000:	learn: 0.0247290	test: 0.1568647	best: 0.1567780 (1989)	total: 6.85s	remaining: 27.4s
3000:	learn: 0.0092830	test: 0.1598822	best: 0.1564179 (2187)	total: 10.2s	remaining: 23.9s
Stoppe

In [8]:
# generate a submission file
test_probs = np.mean(test_probs, axis=0)
sub_df = pd.DataFrame(
    {"Id": test_df.Id, "Class_0": test_probs[:, 0], "Class_1": test_probs[:, 1]}
)
sub_df.to_csv("submission.csv", index=False)
